python: can't open file '__version__': [Errno 2] No such file or directory


In [1]:
import pandas as pd 
import numpy as np
import pickle
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import shap

from pycaret.regression import *

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16})
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = 'lightgrey'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pop_spotify = pd.read_pickle('data/spotify_df.pkl')

In [3]:
pop_spotify.shape

(1927392, 22)

In [4]:
# pop_spotify2 = pop_spotify[['popularity','duration_ms','explicit','danceability','energy','key','mode', 'speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
pop_spotify2 = pop_spotify.sample(frac=1, random_state=786).reset_index(drop=True)
pop_spotify_samp = pop_spotify2[:350000]

In [10]:
corr_matrix = pop_spotify_samp.corr()
corr_matrix['popularity']

popularity          1.000000
duration_ms        -0.088306
explicit            0.221257
danceability        0.160501
energy             -0.038291
key                -0.002842
loudness            0.124566
mode               -0.016648
speechiness         0.089533
acousticness       -0.037630
instrumentalness   -0.148958
liveness           -0.098719
valence            -0.037981
tempo              -0.006913
time_signature      0.017958
year                0.298076
month               0.018001
Name: popularity, dtype: float64

In [5]:
reg1 = setup(data=pop_spotify_samp, target='popularity', train_size=0.80,
            ignore_features=['id','name','artists','id_artists'],
            normalize=True,
            transformation=True, transformation_method='yeo-johnson',
            transform_target=True,
            remove_outliers=True,
            remove_multicollinearity=True,
            log_experiment=True,experiment_name='spotify popularity')
            

,Description,Value
0,session_id,1123
1,Target,popularity
2,Original Data,"(350000, 22)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(266004, 47)"


In [13]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.0661,13.5899,3.6862,0.9020,0.0768,0.0248,90.5770
rf,Random Forest Regressor,1.2435,16.5631,4.0693,0.8806,0.0839,0.0289,85.2430
dt,Decision Tree Regressor,1.5041,27.6673,5.2592,0.8005,0.1024,0.0334,1.2550
knn,K Neighbors Regressor,2.1085,39.9173,6.3176,0.7122,0.1192,0.0471,0.8780
lightgbm,Light Gradient Boosting Machine,5.8931,70.9192,8.4211,0.4887,0.1507,0.1073,1.0870
gbr,Gradient Boosting Regressor,6.6507,90.7351,9.5252,0.3459,0.1706,0.1227,24.6880
ridge,Ridge Regression,7.6633,111.7658,10.5717,0.1943,0.1899,0.1423,0.0630
br,Bayesian Ridge,7.6625,111.7665,10.5717,0.1943,0.1899,0.1423,1.2990
lr,Linear Regression,7.6707,111.9431,10.5800,0.1930,0.1900,0.1424,0.3420
lar,Least Angle Regression,7.6884,112.2053,10.5924,0.1910,0.1902,0.1426,0.0770


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=335, verbose=0, warm_start=False)

In [5]:
et = create_model('et')

NameError: name '_all_models_internal' is not defined

In [21]:
evaluate_model(et)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [19]:
plot_model(et, plot='vc', save=True)

'Validation Curve.png'

In [39]:
# tuned_et = tune_model(et, search_library='scikit-learn')

In [8]:
dt = create_model('dt')
rf = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.2292,17.2381,4.1519,0.8760,0.0858,0.0290
1,1.2057,16.7426,4.0918,0.8790,0.0847,0.0285
2,1.2352,17.0832,4.1332,0.8754,0.0850,0.0289
3,1.2085,16.6751,4.0835,0.8805,0.0842,0.0283
4,1.2429,17.5808,4.1930,0.8730,0.0866,0.0294
5,1.2356,17.1533,4.1417,0.8764,0.0858,0.0291
6,1.2415,17.5691,4.1916,0.8733,0.0862,0.0292
7,1.1958,16.3425,4.0426,0.8815,0.0835,0.0281
8,1.2536,17.7719,4.2157,0.8720,0.0865,0.0295
9,1.2251,16.8563,4.1056,0.8767,0.0843,0.0286


In [9]:
bagged_dt = ensemble_model(dt)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.2876,18.5700,4.3093,0.8665,0.0878,0.0300
1,1.2519,17.6189,4.1975,0.8726,0.0859,0.0292
2,1.2970,18.4013,4.2897,0.8658,0.0871,0.0300
3,1.2539,17.9432,4.2359,0.8714,0.0861,0.0291
4,1.3005,18.7707,4.3325,0.8644,0.0885,0.0304
5,1.2694,17.8905,4.2297,0.8711,0.0866,0.0296
6,1.2885,18.5592,4.3080,0.8662,0.0875,0.0299
7,1.2346,17.2244,4.1502,0.8751,0.0848,0.0287
8,1.3001,18.7374,4.3287,0.8650,0.0879,0.0303
9,1.2766,18.0013,4.2428,0.8683,0.0860,0.0295


In [ ]:
boosted_et = ensemble_model(et, method = 'Boosting')

IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:51:46
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


In [8]:
final_et = finalize_model(et)

In [9]:
predict = predict_model(final_et, data=pop_spotify)


In [10]:
predict[['popularity','Label','artists','name']].sample(frac=1, random_state=786).head(100)

,popularity,Label,artists,name
79805,66,66.000000,['Leona Lewis'],Run
79412,71,71.041974,['Julieta Venegas'],Limón y Sal
83346,64,64.000000,"['Swedish House Mafia', 'Pharrell Williams']",One (Your Name) - Radio Edit
91909,79,79.000000,"['Ariana Grande', 'Social House']",boyfriend (with Social House)
81659,60,60.000000,['La Nueva Luna'],Te Vas a Arrepentir
...,...,...,...,...
83288,63,63.000000,"['Joe ""Bean"" Esposito']",You're The Best
81939,60,60.000000,['Glenn Fredly'],Terserah
75282,72,72.000000,['Nirvana'],About A Girl
79349,74,74.129860,['John Mayer'],Gravity


In [11]:
saved_final_et

NameError: name 'saved_final_et' is not defined

In [1]:
# true = predict['popularity']
# predicted = predict['Label']

# plt.figure(figsize=(15,15))
# plt.scatter(true,predicted, c='crimson')

# p1 = max(max(predicted), max(true))
# p2 = min(min(predicted), min(true))
# plt.plot([p1, p2], [p1, p2], 'b-')
# plt.xlabel('True Values')
# plt.ylabel('Predictions')
# plt.axis('equal')
# plt.show()

In [36]:
plot_model(predict, plot= 'residuals')

KeyboardInterrupt: 

In [77]:
y = pop_spotify_samp.popularity
X = pop_spotify_samp.drop(['popularity'], axis=1)